#영화 리뷰 감성 분석
목표: 나이브 베이즈 알고리즘을 사용하여 영화 리뷰가 긍정적인지 부정적인지 판별한다.

##사용할 IMDb 데이터셋의 특징
데이터 파일은 25000개의 train 데이터와 25000개의 test 데이터로 분류되어 있다.  
train과 test 각각 pos와 neg 폴더로 데이터가 나뉘어 들어가 있어 label을 파악할 수 있다.  
train 폴더 내 unsup 폴더의 데이터들은 unsupervised learning을 위해 label이 없다.  
unsup 폴더의 데이터는 마지막에 새로운 데이터 예측을 할 때 사용될 예정이다.  

데이터 형식은 [id]_[rating.txt]으로 이루어져 있다.  
예를 들어 [test/pos/200_8.txt] 텍스트 파일은 IMDb에서 고유 ID 200과 rating 등급이 8/10인 positive label 텍스트 데이터이다.  
[train/unsup/]은 rating이 없이 생략된다.  
.feat 파일은 .vocab 파일에 대한 already-tokenized bag of words (BoW) features가 들어가 있어, 여기서는 사용하지 않겠다.

In [ ]:
# IMDb 데이터셋 다운로드 및 로드
# https://ai.stanford.edu/~amaas/data/sentiment/
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2023-10-05 11:18:53--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  23.1MB/s    in 5.5s    

2023-10-05 11:18:59 (14.5 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



##데이터 불러오기

In [ ]:
import pandas as pd
import os

# 데이터 경로 설정
data_dir = 'aclImdb'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# 각 데이터셋(train, test)의 긍정(positive) 및 부정(negative) 리뷰 파일 경로 가져오기
positive_train_files = [os.path.join(train_dir, 'pos', filename) for filename in os.listdir(os.path.join(train_dir, 'pos'))]
negative_train_files = [os.path.join(train_dir, 'neg', filename) for filename in os.listdir(os.path.join(train_dir, 'neg'))]
positive_test_files = [os.path.join(test_dir, 'pos', filename) for filename in os.listdir(os.path.join(test_dir, 'pos'))]
negative_test_files = [os.path.join(test_dir, 'neg', filename) for filename in os.listdir(os.path.join(test_dir, 'neg'))]

# 파일 내용을 읽어서 데이터프레임으로 만들기
def read_files(file_list, label):
    data = []
    for file_path in file_list:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            data.append([text, label])
    return data

positive_train_data = read_files(positive_train_files, 'positive')
negative_train_data = read_files(negative_train_files, 'negative')
positive_test_data = read_files(positive_test_files, 'positive')
negative_test_data = read_files(negative_test_files, 'negative')

# 데이터프레임으로 변환
# train과 test를 그대로 1:1으로 하지 않고, 8:2 비율로 만들고자 일단 전체를 합침
df = pd.DataFrame(positive_train_data + negative_train_data + positive_test_data + negative_test_data, columns=['review', 'sentiment'])

df.head()

,review,sentiment
0,It isn't always easy to explain what a movie i...,positive
1,A mock documentary about a pair of Canadian pr...,positive
2,This movie is inspiring to anyone who is or ha...,positive
3,"I cried my heart out, watching this movie. I h...",positive
4,This was a wonderful film. How these women tri...,positive


##Preprocessing
결측치가 없다는 것이 확인되었다.

In [ ]:
# 결측치 확인
print(df.shape)
print(df.isnull().sum())

(50000, 2)
review       0
sentiment    0
dtype: int64


##Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X = df['review']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('X_train: ',X_train.shape)
print(X_train[0:3])
print()
print('y_train: ',y_train.shape)
print(y_train[0:3])

X_train:  (40000,)
39087    I saw the *star* of this movie on The Daily Sh...
30893    While many pass this off as a B movie it is, i...
45278    Trading on the success of the 1975 hit, this f...
Name: review, dtype: object

y_train:  (40000,)
39087    negative
30893    positive
45278    negative
Name: sentiment, dtype: object


##모델 학습
텍스트 데이터 벡터화 및 모델 학습

영화 리뷰가 긍정적인지 부정적인지 판별하는 것은 이진 분류이기 때문에, 베르누이(Bernoulli) 나이브 베이즈 모델을 사용하는 것이 적절하다고 판단했다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline

# CountVectorizer와 BernoulliNB 사용 파이프라인
# 이진 분류이므로 BernoulliNB 사용
model = Pipeline([
    ('vect', TfidfVectorizer(stop_words="english")),
    ('nb', BernoulliNB()),
])

# 모델 학습
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

##Analysis

Accuracy는 약 85.04%가 나왔고, precision과 recall 등이 포함된 classification report와 confusion matrix는 아래와 같다.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 성능 평가
# 정확도 및 분류 보고서 출력
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.8504

Classification Report:
               precision    recall  f1-score   support

    negative       0.82      0.89      0.85      4945
    positive       0.88      0.81      0.85      5055

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000


Confusion Matrix:
 [[4388  557]
 [ 939 4116]]


##새로운 데이터 예측
train과 test 데이터에 포함되지 않은 unsup 데이터를 새로운 데이터로 사용했다. 텍스트 파일은 12345_0을 사용했으나, 다른 파일을 사용해도 무방하다.

In [ ]:
# 임의의 새로운 데이터 파일 경로
new_data_path = 'aclImdb/train/unsup/12345_0.txt'

# 파일 내용 읽기
with open(new_data_path, 'r', encoding='utf-8') as file:
    new_data = [file.read()]

# 모델을 사용하여 새로운 데이터에 대한 예측 수행
new_data_pred = model.predict(new_data)

# 예측 결과 출력
print('new data')
print(new_data)
print("새로운 데이터 예측 결과:", new_data_pred)

new data
["<br /><br />Definitely Nagesh Kukunoor's best effort to date.<br /><br />Frankly i went in expecting another average fare from Nagesh, cos i havent liked his earlier films. I thought Hyderabad Blues was hugely hyped, not to mention shoddy. Rockford was a good idea gone horribly wrong and the only reason i sat through Bollywood calling was Om Puri (outstanding).<br /><br />With this background i set out to catch 3 Deewarein. Actually the glowing reviews (which as an afterthought i think were completely justified)pushed me further towards the theatre. Not to mention Juhi Chawla who i have always believed is one of our most underrated actresses. And what an experience it was!<br /><br />3 deewarein is a very good film. Why? Let me give you 2 reasons:<br /><br />1. The script is near to brilliant. Though the narrative is slow paced, your interest never wanes. The story unfolds slow and leads to a mind-blowing climax. Least expected but a shocker of a climax. Like my buddy aptly 